In [1]:
import os
import sys

# append PYTHONPATH to load extensions
sys.path.append('fastaugment')
sys.path.append('sigmoid_like_tf_op')

# load TensorFlow
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# import other modules
import numpy
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.data.experimental import AUTOTUNE
import model
from fast_augment import augment, center_crop
from sigmoid_like import sigmoid_like

In [2]:
# Setup the model

model = model.make_model(input_size=385, activation=sigmoid_like, num_classes=120)

# Load pretrained 120-class model
model.load_weights('model.h5')

# Make all its layers non-trainable
for layer in model.layers:
    layer.trainable = False

# Replace last dense layer by a new one with 40 units on output
dense_input = model.get_layer('Dense').input
new_dense = tf.keras.layers.Dense(units=40, name='Dense')(dense_input)
model = tf.keras.models.Model(inputs=model.input, outputs=new_dense)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
          loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
          metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 385, 385, 3) 0                                            
__________________________________________________________________________________________________
b0-conv (Conv2D)                (None, 191, 191, 32) 2400        input[0][0]                      
__________________________________________________________________________________________________
b0-conv-bn (BatchNormalization) (None, 191, 191, 32) 128         b0-conv[0][0]                    
__________________________________________________________________________________________________
b0-conv-act (Activation)        (None, 191, 191, 32) 0           b0-conv-bn[0][0]                 
_______________________________________________________________________________________

In [4]:
# Load dataset
image_size = model.layers[0].input_shape[0][1]

(train_set, test_set), info = tfds.load(
    'oxford_iiit_pet',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True)

crop_size = 6 * image_size // 5

train_set = train_set.shuffle(info.splits['train'].num_examples)
train_set = train_set.map(lambda x, y: (center_crop(x, size=[crop_size, crop_size]), y))
train_set = train_set.batch(64)
train_set = train_set.cache()
train_set = train_set.prefetch(AUTOTUNE)
train_set = train_set.map(lambda x, y: augment(tf.cast(x, tf.uint8),
                                               tf.one_hot(y, 40),
                                               output_size=[image_size, image_size],
                                               prescale=image_size/crop_size,
                                               mixup=0.5,
                                               perspective=20))

test_set = test_set.map(lambda x, y: (center_crop(x, size=[image_size, image_size]) / 255,
                                      tf.one_hot(y, 40)))
test_set = test_set.batch(64)
test_set = test_set.prefetch(AUTOTUNE)

In [ ]:
# Sample augmented images from the training set
it = train_set.take(1).unbatch().as_numpy_iterator()

plt.figure(figsize=(12,12))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image, _ = next(it)
    plt.imshow(image)
plt.show()

In [5]:
# FFEEEEAAAAATTTTTT

# define LR schedule
def lr_scheduler(epoch, _, initial_lr=0.1, cliff=10, step=10):
    step = max(0, (epoch - cliff) // step + 1)
    factor = 0.5
    lr = initial_lr * (factor ** step)
    return lr

model.fit(train_set,
          validation_data=test_set,
          validation_freq=1,
          epochs=100,
          callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler)])

Epoch 1/100
58/58 [==============================] - 31s 535ms/step - loss: 2.3585 - categorical_accuracy: 0.5174 - val_loss: 1.0933 - val_categorical_accuracy: 0.7817
Epoch 2/100
58/58 [==============================] - 26s 452ms/step - loss: 1.3466 - categorical_accuracy: 0.7481 - val_loss: 0.7210 - val_categorical_accuracy: 0.8310
Epoch 3/100
58/58 [==============================] - 28s 475ms/step - loss: 1.0935 - categorical_accuracy: 0.7837 - val_loss: 0.6093 - val_categorical_accuracy: 0.8422
Epoch 4/100
58/58 [==============================] - 29s 496ms/step - loss: 1.0391 - categorical_accuracy: 0.8071 - val_loss: 0.5620 - val_categorical_accuracy: 0.8430
Epoch 5/100
58/58 [==============================] - 28s 489ms/step - loss: 0.9937 - categorical_accuracy: 0.8125 - val_loss: 0.5195 - val_categorical_accuracy: 0.8479
Epoch 6/100
58/58 [==============================] - 29s 497ms/step - loss: 0.9400 - categorical_accuracy: 0.8321 - val_loss: 0.4920 - val_categorical_accuracy:

58/58 [==============================] - 30s 520ms/step - loss: 0.7965 - categorical_accuracy: 0.8734 - val_loss: 0.4256 - val_categorical_accuracy: 0.8659
Epoch 50/100
58/58 [==============================] - 30s 522ms/step - loss: 0.8168 - categorical_accuracy: 0.8723 - val_loss: 0.4235 - val_categorical_accuracy: 0.8662
Epoch 51/100
58/58 [==============================] - 30s 526ms/step - loss: 0.7890 - categorical_accuracy: 0.8783 - val_loss: 0.4225 - val_categorical_accuracy: 0.8681
Epoch 52/100
58/58 [==============================] - 30s 520ms/step - loss: 0.8179 - categorical_accuracy: 0.8696 - val_loss: 0.4231 - val_categorical_accuracy: 0.8681
Epoch 53/100
58/58 [==============================] - 30s 524ms/step - loss: 0.7795 - categorical_accuracy: 0.8769 - val_loss: 0.4241 - val_categorical_accuracy: 0.8670
Epoch 54/100
58/58 [==============================] - 30s 522ms/step - loss: 0.8079 - categorical_accuracy: 0.8755 - val_loss: 0.4239 - val_categorical_accuracy: 0.8667

58/58 [==============================] - 30s 524ms/step - loss: 0.8393 - categorical_accuracy: 0.8690 - val_loss: 0.4204 - val_categorical_accuracy: 0.8684
Epoch 98/100
58/58 [==============================] - 30s 519ms/step - loss: 0.7933 - categorical_accuracy: 0.8720 - val_loss: 0.4205 - val_categorical_accuracy: 0.8684
Epoch 99/100
58/58 [==============================] - 30s 524ms/step - loss: 0.7884 - categorical_accuracy: 0.8666 - val_loss: 0.4205 - val_categorical_accuracy: 0.8684
Epoch 100/100
58/58 [==============================] - 30s 517ms/step - loss: 0.8005 - categorical_accuracy: 0.8774 - val_loss: 0.4206 - val_categorical_accuracy: 0.8684


In [ ]:
# Run validation pass
model.evaluate(test_set)